# Model Building

## Import Dependencies

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import os
from glob import glob
import argparse
from tqdm import tqdm
from sklearn.neighbors import BallTree
from sklearn.preprocessing import RobustScaler
from xgboost import XGBRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
import optuna
from optuna.samplers import TPESampler
import logging

import warnings
warnings.filterwarnings('ignore')

## Define Files and Directory Paths

In [2]:
# Main directory containing raw raster and vector datasets
MAIN_DATA_DIR = r'/beegfs/halder/DATA'

# GitHub-linked project directory where processed data and results are stored
PROJECT_DATA_DIR = r'/beegfs/halder/GITHUB/Landscape-Analysis/data'

# Temporary directory used for storing intermediate files
TEMP_DIR = os.path.join(PROJECT_DATA_DIR, 'TEMP')

# Output directory
OUT_DIR = os.path.join(PROJECT_DATA_DIR, 'OUTPUT')

## Load Hexagonal Grid for Germany

In [3]:
# Define the distance and epsg
DISTANCE = 5
EPSG = 25832
CROP = 'WW'
CROP_CODE = 1110

# Path to grid shapefile
GRID_PATH = os.path.join(PROJECT_DATA_DIR, 'VECTOR', f'DE_Hexbins_{DISTANCE}sqkm_EPSG_{EPSG}.shp')

# Load grid as a GeoDataFrame and retain relevant columns
grids_gdf = gpd.read_file(GRID_PATH)
grids_gdf = grids_gdf[['id', 'geometry']]
grids_gdf['id'] = grids_gdf['id'].astype(int)

print('Grids Shape:', grids_gdf.shape)
print('Successfully read the grids!')

Grids Shape: (17198, 2)
Successfully read the grids!


## Load the Data

In [4]:
# Load the data for model training
data = pd.read_csv(os.path.join(OUT_DIR, f'Landscape_Data_{DISTANCE}KM.csv'))

# Drop the geometry column
data.drop(columns=['geometry'], inplace=True)

print(data.shape)
data.head()

(313120, 52)


,id,NUTS_ID,year,landscape_shannon_diversity_index,landscape_patch_density,landscape_euclidean_nearest_neighbor_mn,productive_prop_landscape,productive_patch_density,productive_edge_density,productive_shape_index,...,tmin_mean,tmin_std,tmax_max,tmax_mean,tmax_std,rad_min,rad_max,rad_mean,rad_std,distributed_yield
0,89,DEA29,2001,1.156884,49.940132,42.874456,40.181865,16.977797,84.577557,22.417621,...,6.452202,5.309190,31.29375,13.913024,7.332923,1.731437,30.001625,9.851031,7.594433,9.492277
1,89,DEA29,2002,1.156884,49.940132,42.874456,40.181865,16.977797,84.577557,22.417621,...,6.428998,5.828641,34.18750,14.153472,7.251234,1.730438,29.672750,9.935940,7.149082,8.554052
2,89,DEA29,2003,1.156884,49.940132,42.874456,40.181865,16.977797,84.577557,22.417621,...,5.647917,6.392592,35.52500,14.464478,8.405143,1.730438,28.079750,10.905115,8.027029,8.513234
3,89,DEA29,2004,1.156884,49.940132,42.874456,40.181865,16.977797,84.577557,22.417621,...,5.048927,5.381756,30.71875,13.271401,7.016700,1.730438,28.814375,10.000559,7.281448,8.807802
4,89,DEA29,2005,1.156884,49.940132,42.874456,40.181865,16.977797,84.577557,22.417621,...,5.628146,6.027868,33.76250,13.608452,8.250902,1.734000,29.224062,10.047520,7.573995,9.068336


## Leave-Location-and-Time-Out with Moving Window (LLTO-MW) Split

In [5]:
def leave_location_and_time_out_moving_window(
    data,
    year_col,
    space_col,
    train_years=10,
    test_years=1,
    test_frac=0.3,
    random_state=42
):
    """
    Leave-Location-and-Time-Out with Moving Window (LLTO-MW)

    Implements a generalized LLTO cross-validation strategy:
    - Trains on a moving window of fixed training years
    - Leaves out a random 30% of locations for testing in the next test_year(s)
    
    Each split:
    - Trains on `train_years` using 70% of locations
    - Tests on the immediately following `test_years` using the remaining 30% of locations

    Parameters:
    ----------
    data : pd.DataFrame
        Input dataframe containing temporal and spatial columns.
    year_col : str
        Column name representing the year (int or datetime).
    space_col : str
        Column name representing the spatial unit (e.g., district, grid).
    train_years : int, default=10
        Number of consecutive years to use for training.
    test_years : int, default=1
        Number of consecutive years to use for testing (after training window).
    test_frac : float, default=0.3
        Fraction of locations to leave out for testing.
    random_state : int, default=42
        Seed for reproducibility of location sampling.

    Returns:
    -------
    splits : list of (train_indices, test_indices)
        Index tuples for training and testing data in each LLTO-MW fold.
    """
    data = data.copy()
    splits = []

    all_years = sorted(data[year_col].unique())
    all_locations = np.array(data[space_col].unique())
    rng = np.random.default_rng(seed=random_state)

    print("\nLeave-Location-and-Time-Out with Moving Window (LLTO-MW)\n" + "-" * 60)

    for start_idx in range(len(all_years) - train_years - test_years + 1):
        train_year_start = all_years[start_idx]
        train_year_end = all_years[start_idx + train_years - 1]
        test_year_start = all_years[start_idx + train_years]
        test_year_end = test_year_start + test_years - 1

        # Random 70-30 location split
        shuffled_locations = rng.permutation(all_locations)
        split_idx = int((1 - test_frac) * len(shuffled_locations))
        train_locs = shuffled_locations[:split_idx]
        test_locs = shuffled_locations[split_idx:]

        train_mask = (
            data[year_col].between(train_year_start, train_year_end) &
            data[space_col].isin(train_locs)
        )
        test_mask = (
            data[year_col].between(test_year_start, test_year_end) &
            data[space_col].isin(test_locs)
        )

        train_idx = data[train_mask].index
        test_idx = data[test_mask].index

        if len(train_idx) > 0 and len(test_idx) > 0:
            splits.append((list(train_idx), list(test_idx)))
            print(f"Train: {train_year_start}-{train_year_end} | Test: {test_year_start}-{test_year_end} | "
                  f"Train Locs: {len(train_locs)} | Test Locs: {len(test_locs)} "
                  f"({len(train_idx)} train, {len(test_idx)} test)")

    return splits

In [6]:
def robust_scale_train_test(train_df, test_df):
    """
    Applies RobustScaler to train and test DataFrames based on train data statistics.
    NaN values are ignored during fitting and preserved in output.

    Parameters:
    - train_df (pd.DataFrame): Training dataset
    - test_df (pd.DataFrame): Testing dataset

    Returns:
    - scaled_train (pd.DataFrame): Robust-scaled training data
    - scaled_test (pd.DataFrame): Robust-scaled test data
    """

    # Initialize the scaler
    scaler = RobustScaler()

    # Fit only on non-NaN values in training data
    scaler.fit(train_df.dropna())

    # Transform while preserving original index and column names
    scaled_train = pd.DataFrame(
        scaler.transform(train_df),
        index=train_df.index,
        columns=train_df.columns
    )

    scaled_test = pd.DataFrame(
        scaler.transform(test_df),
        index=test_df.index,
        columns=test_df.columns
    )

    # Preserve original NaN values
    scaled_train[train_df.isna()] = pd.NA
    scaled_test[test_df.isna()] = pd.NA

    return scaled_train, scaled_test

In [7]:
# Extract the group index
group_index = leave_location_and_time_out_moving_window(
    data=data, 
    year_col='year',
    space_col='NUTS_ID',
    train_years=10,
    test_years=1,
    test_frac=0.3,
    random_state=42
)

# Define the columns to be droppend from 'X'
cols_to_be_dropped = ['id', 'NUTS_ID', 'year', 'distributed_yield']


Leave-Location-and-Time-Out with Moving Window (LLTO-MW)
------------------------------------------------------------
Train: 2001-2010 | Test: 2011-2011 | Train Locs: 250 | Test Locs: 108 (96922 train, 4155 test)
Train: 2002-2011 | Test: 2012-2012 | Train Locs: 250 | Test Locs: 108 (100268 train, 3817 test)
Train: 2003-2012 | Test: 2013-2013 | Train Locs: 250 | Test Locs: 108 (94787 train, 4400 test)
Train: 2004-2013 | Test: 2014-2014 | Train Locs: 250 | Test Locs: 108 (89986 train, 4826 test)
Train: 2005-2014 | Test: 2015-2015 | Train Locs: 250 | Test Locs: 108 (95284 train, 3933 test)
Train: 2006-2015 | Test: 2016-2016 | Train Locs: 250 | Test Locs: 108 (98566 train, 3845 test)
Train: 2007-2016 | Test: 2017-2017 | Train Locs: 250 | Test Locs: 108 (97713 train, 3905 test)
Train: 2008-2017 | Test: 2018-2018 | Train Locs: 250 | Test Locs: 108 (97163 train, 3790 test)
Train: 2009-2018 | Test: 2019-2019 | Train Locs: 250 | Test Locs: 108 (94006 train, 4132 test)
Train: 2010-2019 | Test: 

## XGBoost Model Implementation

In [8]:
# Define the Optuna objective function
def objective(trial):
    params = {
        "max_depth": trial.suggest_int("max_depth", 4, 20),
        "learning_rate": trial.suggest_loguniform("learning_rate", 0.01, 1.0),
        "n_estimators": trial.suggest_int("n_estimators", 50, 1000),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "gamma": trial.suggest_loguniform("gamma", 0.001, 5),
        "subsample": trial.suggest_loguniform("subsample", 0.1, 1.0),
        "colsample_bytree": trial.suggest_loguniform("colsample_bytree", 0.1, 1.0),
        "reg_alpha": trial.suggest_loguniform("reg_alpha", 0.0001, 10),
        "reg_lambda": trial.suggest_loguniform("reg_lambda", 0.0001, 10)
    }

    # Log trial parameters
    logger.info(f"\n\n\nTrial {trial.number} - Params: {params}")

    model = XGBRegressor(**params, tree_method="hist", predictor="gpu_predictor", device="cuda")
    
    losses = []
    for fold, (train_index, test_index) in enumerate(group_index):
        X_train = data.loc[train_index].drop(cols_to_be_dropped, axis=1)
        y_train = data.loc[train_index]['distributed_yield']

        X_test = data.loc[test_index].drop(cols_to_be_dropped, axis=1)
        y_test = data.loc[test_index]['distributed_yield']

        # Standardize the data
        X_train_scaled, X_test_scaled = robust_scale_train_test(X_train, X_test)

        # Fit the model
        model.fit(X_train_scaled, y_train)

        # Predict on the test set
        test_preds = model.predict(X_test_scaled)

        # Calculate RMSE loss on the current test set
        loss = np.sqrt(mean_squared_error(y_test, test_preds))
        losses.append(loss)

        # Log fold results
        logger.info(f"\tFold {fold}, Test MSE: {loss:.4f}")
        
    # Calculate average loss over all folds
    average_loss = np.mean(losses)

    # Log final trial result
    logger.info(f"\tTrial {trial.number} - Average Test RMSE: {average_loss:.4f}\n")

    # Return the average score
    return average_loss

In [9]:
# Configure logging to save logs with timestamps
logging.basicConfig(
    filename=os.path.join(OUT_DIR, f"xgboost_{DISTANCE}KM.log"),  # Log file name
    level=logging.INFO,  # Capture detailed logs
    format="%(asctime)s - %(levelname)s - %(message)s",  # Includes timestamp
    datefmt="%Y-%m-%d %H:%M:%S",  # Custom timestamp format
    filemode="w"  # "w" = overwrite each run, use "a" to append
)
logger = logging.getLogger()
optuna.logging.enable_propagation()  # Propagate logs to the root logger.
optuna.logging.disable_default_handler()  # Stop showing logs in sys.stderr.

sampler = TPESampler(seed=42)
study = optuna.create_study(study_name="xgboost", sampler=sampler, direction="minimize")

logger.info("Start optimization.")
study.optimize(objective, n_trials=100, show_progress_bar=True)

  0%|          | 0/100 [00:00<?, ?it/s]

[I 2025-06-24 00:07:06,047] Trial 0 finished with value: 9916.528604761621 and parameters: {'max_depth': 10, 'learning_rate': 0.7969454818643931, 'n_estimators': 746, 'min_child_weight': 6, 'gamma': 0.003776663327107336, 'subsample': 0.14321698289111517, 'colsample_bytree': 0.1143098387631322, 'reg_alpha': 2.1423021757741068, 'reg_lambda': 0.10129197956845731}. Best is trial 0 with value: 9916.528604761621.
[I 2025-06-24 00:22:25,797] Trial 1 finished with value: 1.1400319233448164 and parameters: {'max_depth': 16, 'learning_rate': 0.010994335574766204, 'n_estimators': 972, 'min_child_weight': 9, 'gamma': 0.00610149136730271, 'subsample': 0.1519934830130981, 'colsample_bytree': 0.1525472945805261, 'reg_alpha': 0.0033205591037519565, 'reg_lambda': 0.042051564509138675}. Best is trial 1 with value: 1.1400319233448164.
[I 2025-06-24 00:27:53,226] Trial 2 finished with value: 1.1497980286237417 and parameters: {'max_depth': 11, 'learning_rate': 0.038234752246751866, 'n_estimators': 631, 'm